# 🤖 Smart Business Assistant - Capstone Project
A customer-facing chatbot using NLP, ML and semantic search.

In [ ]:
!pip install -q sentence-transformers scikit-learn pandas

In [ ]:
import pandas as pd
import re
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [ ]:
data = {
    "intent": ["greeting", "greeting", "goodbye", "ask_hours", "ask_hours", "order_status", "order_status"],
    "example": [
        "Hello",
        "Hi there",
        "Bye",
        "What time do you open?",
        "When are you open?",
        "Where is my order 12345?",
        "Track order ABCD"
    ]
}
df = pd.DataFrame(data)
df

In [ ]:
X = df["example"]
y = df["intent"]
vectorizer = CountVectorizer()
X_vec = vectorizer.fit_transform(X)
clf = LogisticRegression()
clf.fit(X_vec, y)

def predict_intent(text):
    v = vectorizer.transform([text])
    return clf.predict(v)[0]

In [ ]:
def extract_order_id(text):
    match = re.search(r"\b[a-zA-Z0-9]{3,}\b", text)
    return match.group(0) if match else None

In [ ]:
faq_data = {
    "What is your return policy?": "You can return products within 30 days.",
    "How do I track my order?": "Please provide your order ID.",
    "What time are you open?": "We are open from 9 AM to 9 PM every day."
}
model = SentenceTransformer('all-MiniLM-L6-v2')
faq_questions = list(faq_data.keys())
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)

def semantic_faq_answer(user_question):
    query_embedding = model.encode(user_question, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, faq_embeddings)[0]
    top_idx = scores.argmax()
    return faq_data[faq_questions[top_idx]]

In [ ]:
session = {}

def handle_message(text):
    global session
    if session.get("pending") == "order_status":
        oid = extract_order_id(text)
        if oid:
            session.pop("pending")
            return f"Thank you! Order {oid} is currently being processed."
        return "I'm still waiting for your order ID."

    intent = predict_intent(text)
    if intent == "greeting":
        return "Hello! How can I help you today?"
    elif intent == "ask_hours":
        return "We are open from 9 AM to 9 PM every day."
    elif intent == "order_status":
        oid = extract_order_id(text)
        if oid:
            return f"Order {oid} is currently in transit."
        session["pending"] = "order_status"
        return "Can you please provide your order ID?"
    elif intent == "goodbye":
        return "Goodbye! Have a great day!"
    else:
        return semantic_faq_answer(text)